In [3]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [69]:
df = pd.read_csv('weather_noduplicates.csv', delimiter=',')

***
### The goal is to find and fill blanks in the dataset.
#### We iterate over rows and compare current timestamp with the next timestamp to check if there are any gaps in the data.

In [5]:
time_format = '%d.%m.%Y %H:%M:%S'
step_timedelta = 600     # one row is 10 minutes * 60 seconds

def find_missing_rows(df):
    missing_rows = pd.DataFrame()
    total_missing_rows = 0
    
    for idx, row in df.iterrows():
        try:
            next_row = df.iloc[idx+1]
        except IndexError:
            print('\nNo more rows to check. \nLast row was {} {}\n\n'.format(idx, row['Date Time']))

        current_ts = datetime.strptime(row['Date Time'], time_format)
        next_ts = datetime.strptime(next_row['Date Time'], time_format)

        # compare the difference between current timestamp and the next timestamp to our step
        delta_seconds = (next_ts - current_ts).total_seconds()
        if delta_seconds > step_timedelta:
            missing_rows = missing_rows.append(row)
            print('Gap found \n{} {} \n{} {}\n'.format(idx, row['Date Time'], idx+1, next_row['Date Time']))

            # total missing seconds between timestamps is divided by our step 
            rows_missing = int(delta_seconds / step_timedelta) - 1
            print('Timedelta {} sec. {} rows missing \n\n'.format(int(delta_seconds), rows_missing))
            total_missing_rows += rows_missing

    print('Total missing rows : ', total_missing_rows)
    return total_missing_rows, missing_rows
    
total_missing_rows, missing_rows = find_missing_rows(df)

Gap found 
40377 08.10.2009 09:40:00 
40378 08.10.2009 10:10:00

Timedelta 1800 sec. 2 rows missing 


Gap found 
229874 16.05.2013 08:50:00 
229875 16.05.2013 09:10:00

Timedelta 1200 sec. 1 rows missing 


Gap found 
293228 30.07.2014 08:00:00 
293229 30.07.2014 08:20:00

Timedelta 1200 sec. 1 rows missing 


Gap found 
301345 24.09.2014 17:00:00 
301346 25.09.2014 09:00:00

Timedelta 57600 sec. 95 rows missing 


Gap found 
410939 25.10.2016 10:30:00 
410940 28.10.2016 12:50:00

Timedelta 267600 sec. 445 rows missing 



No more rows to check. 
Last row was 420223 01.01.2017 00:00:00


Total missing rows :  544


In [6]:
total_days = 6*365 + 2*366  # 2012 and 2016 are leap years
intervals_in_day = 6 * 24   # interval is ten minutes
total_intervals = intervals_in_day * total_days
total_intervals

420768

In [7]:
# sanity check
total_intervals - df.shape[0] == total_missing_rows

True

***
#### We've found 544 missing rows. The main dataset is from Beutenberg WS. Lets check our data against Saaleaue WS data samples to see if we can fill missing rows using their observations.

In [8]:
# this function selects 10 random rows from df1 and checks 
# the same datetimes are present in df2 to make sure dataframes are from the same timeframe

def dataset_match_random_check(df1, df2):
    max_n = df1.shape[0] if df1.shape[0] < df2.shape[0] else df2.shape[0]  # max index from the smaller df
    indexes = np.random.randint(0, max_n, 10).tolist()
    for i in indexes:
        row = df1.iloc[i]
        try:
            df2.loc[df2['Date Time'] == row['Date Time']]
        except:
            print('Row with datetime {} not found'.format(row['Date Time']))
    print('Check done, all ok')

In [9]:
# download data samples from Max-Plank Institute weather station
# https://www.bgc-jena.mpg.de/wetter/weather_data.html

In [10]:
saale_2009 = pd.read_csv('mpi_saale_2009b.csv', delimiter=',', encoding="ISO-8859-1")
saale_2013 = pd.read_csv('mpi_saale_2013a.csv', delimiter=',', encoding="ISO-8859-1")
saale_2014 = pd.read_csv('mpi_saale_2014b.csv', delimiter=',', encoding="ISO-8859-1")
saale_2016 = pd.read_csv('mpi_saale_2016b.csv', delimiter=',', encoding="ISO-8859-1")
dataset_match_random_check(df, saale_2009)
dataset_match_random_check(df, saale_2013)
dataset_match_random_check(df, saale_2014)
dataset_match_random_check(df, saale_2016)

Check done, all ok
Check done, all ok
Check done, all ok
Check done, all ok


In [11]:
# these are target values to look for to complete the dataset df 
missing_rows  

,Date Time,H2OC (mmol/mol),T (degC),Tdew (degC),Tpot (K),VPact (mbar),VPdef (mbar),VPmax (mbar),max. wv (m/s),p (mbar),rh (%),rho (g/m**3),sh (g/kg),wd (deg),wv (m/s)
40377,08.10.2009 09:40:00,16.13,17.80,13.86,292.37,15.86,4.55,20.41,5.38,983.19,77.70,1169.97,10.10,248.50,1.93
229874,16.05.2013 08:50:00,15.15,16.69,12.74,292.08,14.75,4.28,19.03,3.50,973.59,77.50,1163.42,9.48,54.28,1.78
293228,30.07.2014 08:00:00,21.35,19.71,18.22,294.44,20.96,2.05,23.00,0.60,981.50,91.10,1158.04,13.39,101.80,0.31
301345,24.09.2014 17:00:00,10.18,14.04,6.99,288.49,10.02,6.03,16.05,2.20,984.50,62.44,1189.56,6.36,191.00,1.52
410939,25.10.2016 10:30:00,11.04,9.48,8.33,283.07,10.98,0.89,11.87,2.24,994.67,92.50,1220.83,6.90,144.10,1.53


In [12]:
def find_corresponding_rows(target_value, target_row, n_rows, df):
    new_row = df.loc[df[target_row] == target_value] 
    idx_ = new_row.index[0]
    sliced = df[idx_+1:idx_+n_rows+1]
    return sliced

In [112]:
target_value_1 = '25.10.2016 10:30:04'
target_value_2 = '25.10.2016 10:30:00'
n_rows_missing = 445

In [113]:
# NB: offset is 4 sec
new_data = find_corresponding_rows(target_value_1, 'Date Time', n_rows_missing, saale_2016)

In [114]:
new_data.shape

(445, 31)

In [115]:
def adjust_columns(original_data, new_data):
    original_columns = original_data.columns.tolist()
    new_columns = new_data.columns.tolist()
    
    for column in new_columns:
        if column not in original_columns:
            new_data.drop(column, axis=1, inplace=True)
        
    if len(new_data.columns.tolist()) < len(original_columns):
        for c in original_columns:
            if c not in new_data.columns.tolist():
                new_data = new_data.assign(c=np.nan)

    assert len(new_data.columns.tolist()) == len(original_columns)
    new_data = new_data.reindex(columns=original_columns)
    return new_data

In [116]:
adjusted_data = adjust_columns(df, new_data)
adjusted_data.shape

(445, 15)

In [117]:
def correct_offset(pd_series, offset):
    time_format = '%d.%m.%Y %H:%M:%S'
    for i, value in pd_series.iteritems():
        new_value = datetime.strptime(value, time_format) - offset
        pd_series.loc[i] = datetime.strftime(new_value, time_format)
    return pd_series
    
corrected_data = correct_offset(adjusted_data['Date Time'], timedelta(seconds=4))

In [118]:
# find init position to insert new rows
init_index = df_new.loc[df_new['Date Time'] == target_value_2].index[0]
init_index

411038

In [119]:
df_new = pd.concat([df_new.iloc[:init_index+1], 
                    adjusted_data, 
                    df_new.iloc[init_index+1:]]).reset_index(drop=True)

In [120]:
df_new.shape

(420768, 15)

In [121]:
df_new.shape[0] == total_intervals

True

In [123]:
df_new.to_csv('weather_filled.csv', sep=',', index=False)

In [122]:
find_missing_rows(df_new)


No more rows to check. 
Last row was 420767 01.01.2017 00:00:00


Total missing rows :  0


(0, Empty DataFrame
 Columns: []
 Index: [])